In [1]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round desiderato (1-9)
decisions1 = decisions[decisions["Round"] == 9]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
decisions2 = decisions[decisions["Round"] == 10]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")


print("end")

end


In [2]:
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()

#Creo un DataFrame vuoto nel quale concatenerò i coefficienti di regressione per ogni WA
coeffs = pd.DataFrame()


for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    tmp1 = tmp1.drop("Worker Agent ID_y", 1)
    tmp1 = tmp1.drop("Session ID", 1)
    tmp1 = tmp1.sort_values(by = "ID")
    
    
    #Preparazione e training del modello per ogni wa riguardo al round scelto
    xtmp = tmp1.iloc[:, 0:93]
    ytmp = tmp1.iloc[:, 93]
    
    reg = LinearRegression().fit(xtmp, ytmp)
    
    
    #Salvataggio dei coefficienti in un Dataframe Unico
    var = pd.DataFrame(reg.coef_, columns = [f"{i + 1}"])

    totale = pd.concat([totale, tmp1])
    
    coeffs[f"{i+1}"] =  var
    coeffs.to_csv("../Coefficienti di Regressione/coeffs.csv", index = False)
    
print("end")

end


In [3]:
totale = totale.sort_values(by = "ID")

#Preparazione del dataset per il training
x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]
x

,ID,value1,difficulty1,er1,value2,difficulty2,er2,value3,difficulty3,er3,...,difficulty28,er28,value29,difficulty29,er29,value30,difficulty30,er30,Worker Agent ID,Worker Agent Starting Reputation
0,1521,0,0.0,0,0,0.0,0,5,1,5,...,0.0,0,3,0.6,3,0,0.0,0,11,0.093912
0,1523,0,0.0,0,2,0.4,2,0,0,0,...,0.0,0,0,0.0,0,2,0.4,2,16,0.307397
0,1525,4,0.8,4,2,0.4,2,0,0,0,...,0.0,0,3,0.6,3,0,0.0,0,20,0.319685
0,1527,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,13,0.160180
0,1529,4,0.8,4,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,14,0.313495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1290646,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,16,0.548162
1025,1290648,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,2,0.4,2,17,0.690501
1024,1290650,0,0.0,0,0,0.0,0,5,1,5,...,0.0,0,0,0.0,0,0,0.0,0,18,0.577292
1025,1290652,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,19,0.774562


In [4]:
#Training del modello utilizzando la Regressione Lineare

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.9537148596502453

In [5]:
coeffs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3.501672e-09,4.657867e-09,-4.583316e-09,7.909176e-09,-2.315951e-09,-3.780860e-09,1.558012e-09,-2.301770e-09,-9.807293e-10,5.517315e-09,4.352732e-09,-7.051115e-09,8.995016e-10,-6.684871e-09,-1.240321e-09,-7.357570e-09,5.164707e-10,-1.086733e-09,-2.013439e-09,-3.661344e-09
1,-4.319885e-03,3.584513e-04,-9.758426e-04,-4.669163e-04,-2.003636e-03,-5.877134e-04,-1.076409e-03,-6.426206e-04,-2.099967e-04,-1.270542e-03,-1.024457e-03,-1.598342e-03,-1.625747e-03,-1.595483e-03,-7.302575e-04,-1.370270e-03,-5.265689e-04,-7.440085e-04,-8.421453e-04,-3.001999e-04
2,-8.639770e-04,7.169026e-05,-1.951685e-04,-9.338326e-05,-4.007272e-04,-1.175427e-04,-2.152817e-04,-1.285241e-04,-4.199933e-05,-2.541084e-04,-2.048915e-04,-3.196683e-04,-3.251494e-04,-3.190967e-04,-1.460515e-04,-2.740540e-04,-1.053138e-04,-1.488017e-04,-1.684291e-04,-6.003997e-05
3,-4.319885e-03,3.584513e-04,-9.758426e-04,-4.669163e-04,-2.003636e-03,-5.877134e-04,-1.076409e-03,-6.426206e-04,-2.099967e-04,-1.270542e-03,-1.024457e-03,-1.598342e-03,-1.625747e-03,-1.595483e-03,-7.302575e-04,-1.370270e-03,-5.265689e-04,-7.440085e-04,-8.421453e-04,-3.001999e-04
4,-3.119486e-04,-4.789404e-04,3.129986e-04,1.128281e-03,-1.371324e-03,-6.236773e-04,-3.122843e-04,1.610244e-03,-1.158702e-03,-3.696764e-04,-1.362829e-03,-1.417831e-03,1.305819e-03,1.335096e-03,-3.941140e-04,-1.904979e-03,-4.862238e-04,-9.664129e-04,1.998311e-05,7.327546e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,-3.700383e-03,-1.671468e-03,6.273780e-04,6.928714e-04,2.572757e-03,1.468545e-04,-2.357217e-03,7.003582e-04,-2.593594e-03,-1.353735e-03,-1.696578e-03,-1.459417e-04,2.685684e-04,2.644476e-04,-2.271814e-03,1.396511e-03,1.112548e-03,8.778770e-04,-6.318209e-04,1.802169e-03
89,-7.400766e-04,-3.342937e-04,1.254756e-04,1.385743e-04,5.145515e-04,2.937091e-05,-4.714434e-04,1.400716e-04,-5.187188e-04,-2.707470e-04,-3.393156e-04,-2.918834e-05,5.371368e-05,5.288952e-05,-4.543629e-04,2.793023e-04,2.225096e-04,1.755754e-04,-1.263642e-04,3.604337e-04
90,-3.700383e-03,-1.671468e-03,6.273780e-04,6.928714e-04,2.572757e-03,1.468545e-04,-2.357217e-03,7.003582e-04,-2.593594e-03,-1.353735e-03,-1.696578e-03,-1.459417e-04,2.685684e-04,2.644476e-04,-2.271814e-03,1.396511e-03,1.112548e-03,8.778770e-04,-6.318209e-04,1.802169e-03
91,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [40]:
#Training del modello utilizzando SVR

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)

0.788342762977449